In [ ]:
from youtube_transcript_api import YouTubeTranscriptApi, TranscriptsDisabled
from youtube_transcript_api.formatters import TextFormatter

from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_community.vectorstores import FAISS
from langchain_core.prompts import PromptTemplate

import re 
from typing import Optional, List

In [ ]:
def extract_video_id(url):
    """Extracts the YouTube video ID from a URL.

    Args:
        url: The YouTube URL.

    Returns:
        The video ID, or None if not found.
    """
    # Standard URL: https://www.youtube.com/watch?v=VIDEO_ID
    match = re.search(r"v=([a-zA-Z0-9_-]+)", url)
    if match:
        return match.group(1)

    # Short URL: https://youtu.be/VIDEO_ID
    match = re.search(r"youtu\.be\/([a-zA-Z0-9_-]+)", url)
    if match:
        return match.group(1)
    return None

In [ ]:
url1 = "https://www.youtube.com/watch?v=VMj-3S1tku0&list=PLAqhIrjkxbuWI23v9cThsA9GvCAUhRvKZ"
url2 = "https://youtu.be/dQw4w9WgXcQ"
url3 = "invalid-url"
url4 = "https://www.youtube.com/watch?v=aj_OGw-CQeA&ab_channel=Benjamin-DerDeutschlehrer"

video_id1 = extract_video_id(url1)
video_id2 = extract_video_id(url2)
video_id3 = extract_video_id(url3)
video_id4 = extract_video_id(url4)

print(f"Video ID from url1: {video_id1}")
print(f"Video ID from url2: {video_id2}")
print(f"Video ID from url3: {video_id3}")
print(f"Video ID from url3: {video_id4}")

In [ ]:
def get_transcript(video_id: str, languages: Optional[List[str]] = None):
        """
        Fetches and prints the transcript of a YouTube video, along with the raw transcript data.

        Args:
            video_id: The ID of the YouTube video.
            languages: A list of language codes to prioritize (e.g., ["en", "de", "hi"]).
                If None, the "best" available language is fetched.
        """
        try:
            #  Get the transcript as a list of dictionaries
            transcript_list = YouTubeTranscriptApi.get_transcript(video_id, languages=languages)

            #  Print the raw transcript data (list of dictionaries)
            print("Raw Transcript Data:")
            print(transcript_list)

            # Flatten the transcript to plain text and print it
            transcript_text = " ".join(chunk["text"] for chunk in transcript_list)
            print("\nPlain Text Transcript:")
            print(transcript_text)

        except TranscriptsDisabled:
            print("No captions available for this video.")
        except Exception as e:
            print(f"An error occurred: {e}")

In [ ]:
get_transcript(video_id1, languages=["en", "ge"])

In [ ]:
get_transcript(video_id4, languages=["en", "de"])

In [ ]:
import re
from youtube_transcript_api import YouTubeTranscriptApi, TranscriptsDisabled
from typing import List, Optional

def extract_transcript_from_url(youtube_url: str, languages: Optional[List[str]] = None):
    """
    Extracts the transcript of a YouTube video from its URL,
    along with the raw transcript data.

    Args:
        youtube_url: The full YouTube video URL.
        languages: A list of language codes to prioritize (e.g., ["en", "de", "hi"]).
            If None, the "best" available language is fetched.

    Returns:
        The transcript text as a string, or None on error.
    """
    def extract_video_id(url: str) -> str or None:
        """Extracts the YouTube video ID from a URL."""
        match_v = re.search(r"v=([a-zA-Z0-9_-]+)", url)
        if match_v:
            return match_v.group(1)
        match_be = re.search(r"youtu\.be\/([a-zA-Z0-9_-]+)", url)
        if match_be:
            return match_be.group(1)
        return None

    video_id = extract_video_id(youtube_url)

    if not video_id:
        print(f"Error: Could not extract video ID from URL: {youtube_url}")
        return None

    def get_transcript(video_id: str, languages: Optional[List[str]] = None) -> Optional[str]:
        """
        Fetches the transcript of a YouTube video.

        Args:
            video_id: The ID of the YouTube video.
            languages: A list of language codes to prioritize.
                If None, the "best" available language is fetched.

        Returns:
            The transcript text as a string, or None on error.
        """
        try:
            #  Get the transcript as a list of dictionaries
            transcript_list = YouTubeTranscriptApi.get_transcript(video_id, languages=languages)

            # Flatten the transcript to plain text and return it
            transcript_text = " ".join(chunk["text"] for chunk in transcript_list)
            return transcript_text

        except TranscriptsDisabled:
            print("No captions available for this video.")
            return None
        except Exception as e:
            print(f"An error occurred: {e}")
            return None

    return get_transcript(video_id, languages)


In [ ]:
transcript = extract_transcript_from_url(url1, ['de', 'en'])
transcript_1 = extract_transcript_from_url(url4, ['de', 'en'])

In [ ]:
transcript

In [ ]:
transcript_1

## Step 1b -- Indexing (Text Splitting)

In [ ]:
splitter = RecursiveCharacterTextSplitter(chunk_size = 1000, chunk_overlap = 200)
chunks = splitter.create_documents([transcript])

In [ ]:
len(chunks)

## **Step 1c & 1d - Indexing (Embedding Generation and Storing in Vector Store)**

In [ ]:
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")
vector_store = FAISS.from_documents(chunks, embedding=embeddings)

In [ ]:
vector_store.index_to_docstore_id

In [ ]:
vector_store.get_by_ids(['e87046a7-40e3-4c3f-be13-161955cb671d'])

## Step 2 - Retrieval

In [ ]:
retriever = vector_store.as_retriever(search_type = "similarity", search_kwargs = {"k" : 4})

In [ ]:
retriever

In [ ]:
retriever.invoke("was ist Deep Learning?")

## **Step3 - Augmentation**

In [ ]:
template="""
      You are a helpful assistant.
      Answer ONLY from the following transcript context.
      If the context is insufficient to answer the question, just say "I don't know."
      If possible, cite the specific part of the context that supports your answer.

      Transcript Context:
      {context}

      Question: {question}

      Answer:
    """

In [ ]:
prompt = PromptTemplate(template=template, 
    input_variables = ['context', 'question'])

In [ ]:
prompt

In [ ]:
question          = "is the topic of weil discussed in this video? if yes then what was discussed"
retrieved_docs    = retriever.invoke(question)

In [ ]:
retrieved_docs

In [ ]:
context_text = "\n\n".join(doc.page_content for doc in retrieved_docs)
context_text

In [ ]:
final_prompt = prompt.invoke({"context": context_text, "question": question})

In [ ]:
final_prompt

## **Step4 -- Generation**

In [ ]:
answer = llm.invoke(final_prompt)
print(answer.content)

## **Building a Chain**

In [ ]:
from langchain_core.runnables import RunnableParallel, RunnablePassthrough, RunnableLambda
from langchain_core.output_parsers import StrOutputParser

In [ ]:
def format_docs(retrieved_docs):
    context_text = "\n\n".join(doc.page_content for doc in retrieved_docs)
    return context_text

In [ ]:
parallel_chain = RunnableParallel({
    'context': retriever | RunnableLambda(format_docs),
    'question': RunnablePassthrough()
})

In [ ]:
parallel_chain.invoke('who is Micro Grad')

In [ ]:
parser = StrOutputParser()

In [ ]:
main_chain = parallel_chain | prompt | llm | parser

In [ ]:
main_chain.invoke('What is micrograd')

In [ ]:
main_chain.invoke('Can you summarize the video')